In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install pyts

In [4]:
import scipy.io
import matplotlib.pyplot as plt

data = scipy.io.loadmat('/content/drive/My Drive/DATA.mat')


In [5]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'CHAN', 'Fs', 'H', 'S'])

In [6]:
H = data['H']
print(H.shape)
S = data['S']
print(S.shape)
CHAN = data['CHAN']
print(CHAN.shape)
Fs = data['Fs']
print(Fs.shape)


(14, 1)
(14, 1)
(1, 19)
(1, 1)


In [7]:

H = H.reshape(14)
H.shape

print(H.shape)
print(H[0].shape)
print(H[0][0].shape)


(14,)
(19, 180000)
(180000,)


In [8]:
S = S.reshape(14)

print(S.shape)
print(S[0].shape)
print(S[0][0].shape)


(14,)
(19, 180000)
(180000,)


In [9]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from pyts.image import GramianAngularField
from pyts.image import MarkovTransitionField

normal=[]
for j in range(14):
 for i in range(19):
   X = H[j][i,1:15000]
   X = X.reshape(1,-1)
   
   # MTF transformation
  #  mtf = MarkovTransitionField(image_size=24)
  #  X_mtf = mtf.fit_transform(X)


   gasf = GramianAngularField(image_size=112, method='summation')
   X_gasf = gasf.fit_transform(X)
   normal.append(X_gasf)




In [10]:
for j in range(14):
 for i in range(19):
   X = H[j][i,1:15000]
   X = X.reshape(1,-1)
   gasf = GramianAngularField(image_size=112, method='difference')
   X_gasf = gasf.fit_transform(X)
   normal.append(X_gasf)




In [11]:
schizo=[]
for j in range(14):
 for i in range(19):
   X = S[j][i,1:15000]
   X = X.reshape(1,-1)
   gasf = GramianAngularField(image_size=112, method='summation')
   X_gasf = gasf.fit_transform(X)
   schizo.append(X_gasf)




In [12]:
for j in range(14):
 for i in range(19):
   X = S[j][i,1:15000]
   X = X.reshape(1,-1)
   gasf = GramianAngularField(image_size=112, method='difference')
   X_gasf = gasf.fit_transform(X)
   schizo.append(X_gasf)




In [13]:
#create labels
import numpy as np

n = np.zeros((532,))
s = np.ones((532,))
labels = np.concatenate((n,s), axis = 0)

print(len(labels))
print(np.unique(labels))




1064
[0. 1.]


In [14]:
data = []

for img in normal:
  data.append(img)

for img in schizo:
  data.append(img)


len(data)    

1064

In [15]:
data = np.array(data)
data = data.reshape(-1, 112, 112, 1)
labels = np.array(labels)

In [16]:
#split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.2, 
                                                    random_state=42,
                                                    shuffle=True,
                                                    stratify=labels)



print(X_train.shape)
print(X_test.shape)

(851, 112, 112, 1)
(213, 112, 112, 1)


In [18]:
import tensorflow as tf 
import cv2

def gabor_filter(x):
  x = tf.cast(x, dtype=tf.float32)
  # x = tf.image.rgb_to_grayscale(x)
  params = {'ksize':(3, 3), 'sigma':1.0, 'theta': 0, 'lambd':5.0, 'gamma':0.02}
  kernel = cv2.getGaborKernel(**params)
  kernel = tf.expand_dims(kernel, 2)
  kernel = tf.expand_dims(kernel, 3)
  kernel = tf.cast(kernel, dtype=tf.float32)
  return tf.nn.conv2d(x, kernel, strides=[1, 1, 1, 1], padding='VALID')

gabor_layer = tf.keras.layers.Lambda(gabor_filter)

In [22]:
#define model
import tensorflow as tf 
from numpy import mean
from numpy import std
from scipy.io import loadmat
import numpy as np
from pandas import read_csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv1D, Conv2D
from tensorflow.keras.layers import MaxPooling1D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
from sklearn.model_selection import StratifiedKFold, train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
import cv2



model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Input(shape=(112, 112, 1)))
model.add(gabor_layer)

model.add(Conv2D(8, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu'))
                 
model.add(Conv2D(8, kernel_size=(2, 2), activation='relu'))


model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Conv2D(16, (2, 2), activation='relu'))
model.add(Conv2D(16, (2, 2), activation='relu'))


model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (2, 2), activation='relu'))
model.add(Conv2D(32, (2, 2), activation='relu'))

# model.add(fft_layer)

model.add(Dropout(0.25))

model.add(Conv2D(64, (2, 2), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()
# from keras.utils import plot_model
# plot_model(model)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_1 (Lambda)           (None, 110, 110, 1)       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 108, 108, 8)       80        
                                                                 
 conv2d_19 (Conv2D)          (None, 107, 107, 8)       264       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 53, 53, 8)        0         
 2D)                                                             
                                                                 
 dropout_9 (Dropout)         (None, 53, 53, 8)         0         
                                                                 
 conv2d_20 (Conv2D)          (None, 52, 52, 16)        528       
                                                      

In [23]:
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.001)

#compiling the CNN

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

history = model.fit(X_train, Y_train, epochs=200, batch_size=4, verbose=1, validation_data=(X_test, Y_test))
     

Epoch 1/200
213/213 [==============================] - 12s 13ms/step - loss: 0.6471 - accuracy: 0.6157 - val_loss: 0.5315 - val_accuracy: 0.7559
Epoch 2/200
213/213 [==============================] - 2s 10ms/step - loss: 0.5550 - accuracy: 0.7286 - val_loss: 0.4866 - val_accuracy: 0.7512
Epoch 3/200
213/213 [==============================] - 2s 10ms/step - loss: 0.4899 - accuracy: 0.7450 - val_loss: 0.4473 - val_accuracy: 0.7840
Epoch 4/200
213/213 [==============================] - 2s 10ms/step - loss: 0.4313 - accuracy: 0.7779 - val_loss: 0.4429 - val_accuracy: 0.7653
Epoch 5/200
213/213 [==============================] - 2s 11ms/step - loss: 0.4031 - accuracy: 0.8226 - val_loss: 0.3824 - val_accuracy: 0.8310
Epoch 6/200
213/213 [==============================] - 2s 10ms/step - loss: 0.3268 - accuracy: 0.8437 - val_loss: 0.3646 - val_accuracy: 0.8310
Epoch 7/200
213/213 [==============================] - 2s 11ms/step - loss: 0.3081 - accuracy: 0.8801 - val_loss: 0.3284 - val_accuracy

In [24]:
from sklearn.metrics import classification_report
import numpy as np

y_pred = np.around(model.predict(X_test))
rep = classification_report(Y_test, y_pred)
print(rep)

              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96       107
         1.0       0.99      0.92      0.96       106

    accuracy                           0.96       213
   macro avg       0.96      0.96      0.96       213
weighted avg       0.96      0.96      0.96       213

